# Decorators - Dynamically Alter The Functionality Of Your Functions

- [Video](https://www.youtube.com/watch?v=kr0mpwqttM0&feature=youtu.be)
- [Source code](https://github.com/CoreyMSchafer/code_snippets/blob/master/Decorators/snippets.txt)
- http://ot-note.logdown.com/posts/67571/-decorator-with-without-arguments-in-function-class-form
- https://www.programiz.com/python-programming/decorator

## Closures

https://www.programiz.com/python-programming/closure

A **Closure** is a function object that remembers values in enclosing scopes even if they are not present in memory.

The criteria that must be met to create closure in Python are summarized in the following points.

- We must have a nested function (function inside a function).
- The nested function must refer to a value defined in the enclosing function.
- The enclosing function must return the nested function.

#### When to use closures?

So what are closures good for?

- Closures can avoid the use of global values and provides some form of **data hiding**. It can also provide an **object oriented solution** to the problem.
- When there are few methods (one method in most cases) to be implemented in a class, closures can provide an alternate and more elegant solutions. But when the number of attributes and methods get larger, better implement a class.

All function objects have a __closure__ attribute that returns a tuple of cell objects if it is a closure function.

In [14]:
def outer_function(msg):
    def inner_function():
        print(msg)
    return inner_function

hi_func = outer_function('hi')
bye_func = outer_function('bye')

hi_func()
bye_func()

hi
bye


Decorators in Python make an extensive use of closures as well:

In [15]:
def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        print('wrapper executed this before "{}"'.format(
            original_function.__name__))
        original_function(*args, **kwargs)
    return wrapper_function


def display():
    print('display function ran')


@decorator_function
def display_info(name, age):
    print('display_info ran with arguments ({name}, {age})'.format(
        name=name, age=age))


decorated_function = decorator_function(display)
decorated_function()
display_info('John', 35)

wrapper executed this before "display"
display function ran
wrapper executed this before "display_info"
display_info ran with arguments (John, 35)


In [16]:
# https://youtu.be/FsAPt_9Bf3U?t=13m25s
class DecoratorClass(object):

    def __init__(self, original_function):
        print('Enter {} init'.format(self.__class__.__name__))
        # tie the original_function to the instance of Class
        self.original_function = original_function
        print('Exit {} init'.format(self.__class__.__name__))

    def __call__(self, *args, **kwargs):
        print('Call method before "{}"'.format(self.original_function.__name__))
        self.original_function(*args, **kwargs)
        print('Call method after "{}"'.format(self.original_function.__name__))

@DecoratorClass
def display_info(name, age):
    print('"display_info" ran with arguments ({name}, {age})'.format(
        name=name, age=age))


display_info('John', 35)

Enter DecoratorClass init
Exit DecoratorClass init
Call method before "display_info"
"display_info" ran with arguments (John, 35)
Call method after "display_info"


In [17]:
# Decorators
from functools import wraps
import logging


def my_logger(orig_func):

    logging.basicConfig(filename='{}.log'.format(
        orig_func.__name__), level=logging.INFO)

    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        logging.info(
            'Ran with args: {}, and kwargs: {}'.format(args, kwargs))
        return orig_func(*args, **kwargs)

    return wrapper


def my_timer(orig_func):
    import time

    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print('{} ran in: {} sec'.format(orig_func.__name__, t2))
        return result

    return wrapper

In [18]:
# Practical Examples


def my_logger(orig_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(
        orig_func.__name__), level=logging.INFO)

    def wrapper(*args, **kwargs):
        logging.info(
            'Ran with args: {}, and kwargs: {}'.format(args, kwargs))
        return orig_func(*args, **kwargs)

    return wrapper


def my_timer(orig_func):
    import time

    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print('{} ran in: {} sec'.format(orig_func.__name__, t2))
        return result

    return wrapper


import time

---

## Python Decorator 四種寫法範例 Code

### 沒有參數的 Function 版本

> Note: 使用 closure 去處理 function

In [19]:
def decorate_apple(f):
    def d_f(*args, **kargs):
        print("-- apple before call")
        result = f(*args, **kargs)
        print("-- apple after call")
        return result
    return d_f

@decorate_apple
def print_hello():
    print("hello first time.")

print_hello()

-- apple before call
hello first time.
-- apple after call


### 有參數的 Decorator Function

In [20]:
def decorate_fruit(fruit, rot_level):
    def outer_d_f(f):
        def d_f(*args, **kargs):
            print("-- {} {} before call".format(rot_level, fruit))
            result = f(*args, **kargs)
            print("-- {} {} after call".format(rot_level, fruit))
            return result
        return d_f
    return outer_d_f

@decorate_fruit('banana', 'new')
def print_hello2():
    print("hello 2nd time.")

@decorate_fruit('guava', '50% rot')
def print_hello3():
    print("hello 3th time.")

print_hello2()
print('')
print_hello3()

-- new banana before call
hello 2nd time.
-- new banana after call

-- 50% rot guava before call
hello 3th time.
-- 50% rot guava after call


### 沒有參數版本的 Decorator Class

Note:

- Python 會將被修飾的 function 傳入 `__init__()`，這時就將它存入到 decorator 的 data member `self.f` 。另外再直接覆寫 `__call__()` ，然後在呼叫 `self.f` 的前後作額外的修飾動作。
- 這裡的 `__call__()` 的 signature ，就是跟 被修飾的 function 一樣，所以就加上動態的 `*args`, 跟 `**kargs` 就可以完整 delegate了。

In [21]:
class DecorateAppleClass(object):
    def __init__(self, f):
        self.f = f
    
    def __call__(self, *args, **kargs):
        print("-- apple before call")
        result = self.f(*args, **kargs)
        print("-- apple after call")
        return result


@DecorateAppleClass
def print_hello4():
    print("hello 4th time.")

print_hello4()

-- apple before call
hello 4th time.
-- apple after call


### 帶有參數版本的 Decorator Class


Note: 

- 當有參數版本的 decorator class 在修飾 function 時，function 就不會像沒有參數版本的 decorator class 一樣，會將 function 傳入 `__init__`，而是會傳入 decorator 的參數。function 本體則是會在 call 中被傳入，這是一個比較不一樣的地方。
- 這裡的 `__call__()` 的 signature 則是 `def __call__(self, f)`

In [22]:
class DecorateFruitClass(object):
    def __init__(self, fruit, rot_level):
        self.fruit = fruit
        self.rot_level = rot_level

    def __call__(self, f):
        def d_f(*args, **kargs):
            print("-- {} {} before call".format(self.rot_level, self.fruit))
            result = f(*args, **kargs)
            print("-- {} {} after call".format(self.rot_level, self.fruit))
            return result
        return d_f

@DecorateFruitClass('guava', '80% rot')
def print_hello5():
    print("hello 5th times.")

@DecorateFruitClass('banana', '30% rot')
def print_hello6():
    print("hello 6th times.")

print_hello5()
print('')
print_hello6()

-- 80% rot guava before call
hello 5th times.
-- 80% rot guava after call

-- 30% rot banana before call
hello 6th times.
-- 30% rot banana after call
